In [2]:
!pip install transformers datasets rouge_score
!pip install --upgrade transformers datasets newspaper3k rouge_score lxml lxml_html_clean

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.2 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=f6f165c499597c62eb2728f4c6ab1b0fbf4e60f4d2428f5f14dd33b5f0a307d5
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are i

In [3]:
import pandas as pd
import torch
import time
from newspaper import Article
from transformers import MT5Tokenizer, MT5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
from rouge_score import rouge_scorer



In [5]:
# df.head()

In [6]:
# from google.colab import files

# # Specify the file path
# file_path = 'Processed_Data.csv'  # Update with your file's name

# # Download the file
# files.download(file_path)


from datasets import load_dataset

ds = load_dataset("mbshr/XSUMUrdu-DW_BBC")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.03k [00:00<?, ?B/s]

Ur512-train.csv:   0%|          | 0.00/197M [00:00<?, ?B/s]

Ur512-test.csv:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/72837 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3790 [00:00<?, ? examples/s]

In [7]:
# print(df)

In [8]:
from datasets import DatasetDict

# Split the 'train' set into 90% train and 10% validation
split_dataset = ds["train"].train_test_split(test_size=0.1, seed=42)

# Recreate DatasetDict with new splits
dataset_dict = DatasetDict({
    "train": ds["train"],
    "validation": ds["test"],
    "test": ds["test"]
})

In [9]:
# Assuming 'dataset_dict' is your DatasetDict
train_df = dataset_dict['train'].to_pandas()
test_df = dataset_dict['test'].to_pandas()
eval_df = dataset_dict['validation'].to_pandas()

In [10]:
train_df.head()

,url,Summary,Text
0,/ur/کشمیر-کی-بحث-میں-سعودی-عرب-زیر-بحث-آخر-کیو...,پاکستان میں مسئلہ کشمیر پر بحث اس وقت عروج پر ...,پاکستان نے حالیہ برسوں میں ترکی اور ایران سے ت...
1,/ur/وکی-لیکس-کے-بانی-کے-وارنٹ-گرفتاری-جاری/a-6...,سویڈن کی ایک عدالت نے وکی لیکس نامی ویب سائٹ ک...,جمعرات کو سویڈن میں ایک عدالت کی ایک خاتون جج ...
2,/ur/جرمن-پارلمیان-میں-آج-افغان-مشن-پر-ووٹنگ/a-...,افغانستان میں جاری جرمن مشن میں توسیع کے حوالے...,جرمن حکومت چاہتی ہے کہ افغان مشن میں ایک سال ی...
3,/ur/جیمزبونڈ-سیریز-کی-نئی-فلم-نے-ریکارڈ-توڑ-دئ...,انگریز اداکار ڈینئل کریگ کی نئی باجیمز بانڈ فل...,سونی پیکچرز نے کہا ہے کہ فلم Quantum of Solace...
4,/ur/افریقی-یونین-کی-صدر-پہلی-بارایک-خاتون/a-16...,افریقی یونین میں پہلی بار ایک خاتون کو چیئرپرس...,گزشتہ روز یعنی اتوار14 اکتوبر کو انتخابات کے ...


In [ ]:
train_df = train_df[:10000]

In [ ]:
eval_df = eval_df[:1000]

In [1]:
test_df = test_df[:1000]

NameError: name 'test_df' is not defined

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
# ============ STEP 2: Tokenization and Dataset Preparation ============
# Load urT5 tokenizer
model_name = "google/mt5-small"
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)
# tokenizer = T5Tokenizer.from_pretrained("")

# Tokenization function
# def preprocess_function(examples):
#     inputs = ["summarize: " + text for text in examples["Text"]]
#     model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

#     labels = tokenizer(examples["Summary"], max_length=128, truncation=True, padding="max_length")
#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs

# # Convert to Hugging Face Dataset
# dataset = Dataset.from_pandas(df)
# tokenized_datasets = dataset.map(preprocess_function, batched=True)

# # Split dataset
# train_test_split = tokenized_datasets.train_test_split(test_size=0.1)
# train_dataset = train_test_split["train"]
# test_dataset = train_test_split["test"]

# Tokenization function (adjust field names here)
def preprocess_function(examples):
    inputs = ["summarize: " + text for text in examples["Text"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    labels = tokenizer(examples["Summary"], max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


# dataset_tr = Dataset.from_pandas(df)
# # Apply tokenization directly to both train and test
# train_dataset = train_df.map(preprocess_function)
# # test_dataset = test_df.map(preprocess_function)
# eval_dataset = eval_df.map(preprocess_function)

traindataset = Dataset.from_pandas(train_df)
train_dataset = traindataset.map(preprocess_function, batched=True)

validationdataset = Dataset.from_pandas(eval_df)
validation_dataset = validationdataset.map(preprocess_function, batched=True)




In [ ]:
# train_dataset

In [ ]:
# ============ STEP 3: Fine-tune the Model ============

model = MT5ForConditionalGeneration.from_pretrained(model_name)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=5e-5,
    weight_decay=0.01,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
)

# Train the model
trainer.train()

# Save fine-tuned model
model.save_pretrained("fine_tuned_mt5")
tokenizer.save_pretrained("fine_tuned_mt5")


In [23]:
testdataset = Dataset.from_pandas(test_df)
test_dataset = testdataset.map(preprocess_function, batched=True)



Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [24]:
# ============ STEP 4: Evaluation using ROUGE ============
def compute_rouge(predictions, references):
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    scores = [scorer.score(pred, ref) for pred, ref in zip(predictions, references)]
    avg_scores = {
        "rouge1": sum([s["rouge1"].fmeasure for s in scores]) / len(scores),
        "rouge2": sum([s["rouge2"].fmeasure for s in scores]) / len(scores),
        "rougeL": sum([s["rougeL"].fmeasure for s in scores]) / len(scores),
    }
    return avg_scores

# Generate predictions
def generate_summaries(text_list):
    # Move inputs to the same device as the model
    device = model.device  # Get the device of the model (CPU or GPU)
    inputs = tokenizer(["summarize: " + text for text in text_list],padding=True , return_tensors="pt", max_length=512, truncation=True).to(device)
    summary_ids = model.generate(inputs.input_ids, max_length=128)
    return [tokenizer.decode(ids, skip_special_tokens=True) for ids in summary_ids]

# Evaluate on test set
test_texts = test_dataset["Text"][:10]
test_summaries = test_dataset["Summary"][:10]
generated_summaries = generate_summaries(test_texts)
rouge_scores = compute_rouge(generated_summaries, test_summaries)

print("ROUGE Scores:", rouge_scores)

ROUGE Scores: {'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0}


In [21]:

# ============ STEP 5: Inference ============
def summarize_article(article_text):
    inputs = tokenizer("summarize: " + article_text, return_tensors="pt", max_length=512, truncation=True)
    # Move inputs and model to the same device (GPU if available)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inputs = inputs.to(device)
    model.to(device)  # Ensure model is on the same device as inputs
    summary_ids = model.generate(inputs.input_ids, max_length=128)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Example inference
new_article = "یہ ایک خبر کا مضمون ہے جس کا خلاصہ نکالا جائے گ۔"  # Replace with actual text
generated_summary = summarize_article(new_article)
print("Generated Summary:", generated_summary)

Generated Summary: <extra_id_0> ہے۔ یہ ایکہ ہے۔ یہ ایکہ ہے۔ یہ ایکہ ہے۔ یہ ایکہ ہے۔ یہ ایکہ ہے۔ یہ ایکہ ہے۔ یہ ایکہ ہے۔ یہ ایکہ ہے۔ یہ ایکہ ہے۔ یہ ایکہ ہے۔ یہ ایکہ ہے۔ یہ ایکہ ہے۔ یہ ایکہ ہے۔ یہ ایکہ ہے۔ یہ ایکہ ہے۔ یہ ایکہ ہے۔ یہ ایکہ ہے۔ یہ ایکہ ہے۔
